In [2]:
import torch

if torch.backends.mps.is_available():
    print("MPS memory stats:")
    print("Allocated:", torch.mps.current_allocated_memory() / 1024**2, "MB")


MPS memory stats:
Allocated: 0.0 MB


In [4]:
import numpy as np
loaded_kr_sequences = np.load('ATTest_KR.npy', allow_pickle=True)
print(len(loaded_kr_sequences))
loaded_kr_sequences = loaded_kr_sequences.tolist()
print(len(loaded_kr_sequences))


import numpy as np
loaded_AT_sequences = np.load('ATTest_AT.npy', allow_pickle=True)
print(len(loaded_AT_sequences))
loaded_AT_sequences = loaded_AT_sequences.tolist()
print(len(loaded_AT_sequences))

503
503
503
503


In [ ]:
#!/usr/bin/env python

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import (
    ESM3InferenceClient,
    ESMProtein,
    ESMProteinError,
    LogitsConfig,
    LogitsOutput,
)
from typing import List, Dict, Any, Union

###########################
# 1) LOGIN TO HUGGING FACE
###########################
token = os.environ.get("HF_TOKEN", None)
if token:
    login(token=token)
else:
    print("No HF_TOKEN found in environment variables. If the model is private, login might fail.")

###########################
# 2) LOAD MODEL
###########################
print("Loading ESM3-open model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model: ESM3InferenceClient = ESM3.from_pretrained("esm3-open").to(device)
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
print("Model loaded on device:", next(model.parameters()).device)

EMBEDDING_CONFIG = LogitsConfig(
    sequence=True,          # We want sequence-level outputs (residue-level embeddings)
    return_embeddings=True, # Return final-layer embeddings
    return_hidden_states=True  # Also return all hidden states if needed
)

def embed_sequence(model: ESM3InferenceClient, sequence: str) -> Union[LogitsOutput, ESMProteinError]:
    """
    Given a raw amino acid sequence, returns a LogitsOutput containing:
      - output.embeddings  => final-layer embeddings (shape: [1, seq_len, hidden_dim])
      - output.hidden_states => list of all layer outputs, if needed
    """
    print("Embedding sequence of length:", len(sequence))
    try:
        protein = ESMProtein(sequence=sequence)
        protein_tensor = model.encode(protein)
        output = model.logits(protein_tensor, EMBEDDING_CONFIG)
        print("Embedding complete for one sequence.")
        return output
    except Exception as e:
        print("Error during embedding:", e)
        return ESMProteinError(500, str(e))

def embed_sequences_to_pt(sequences: List[str], output_pt: str = "all_embeddings.pt"):
    """
    Embeds a list of protein sequences and saves all embeddings to one .pt file.
    """
    all_embeddings = []
    meta = []

    for idx, seq in enumerate(sequences, 1):
        print(f"[{idx}/{len(sequences)}] Embedding (len={len(seq)})...")
        seq_out = embed_sequence(model, seq)
        if isinstance(seq_out, ESMProteinError):
            print(f"Error embedding sequence {idx}: {seq_out}")
            continue

        embeddings = seq_out.embeddings.detach().cpu()  # shape [1, seq_len, hidden_dim]
        all_embeddings.append(embeddings)
        meta.append({
            "index": idx,
            "input_length": len(seq),
            "embedding_shape": tuple(embeddings.shape)
        })

    torch.save({"embeddings": all_embeddings, "meta": meta}, output_pt)
    print(f"\n✓ Saved {len(all_embeddings)} sequence embeddings to {output_pt}")

if __name__ == "__main__":
    embed_sequences_to_pt(loaded_kr_sequences, "ATTest_KR.pt")


No HF_TOKEN found in environment variables. If the model is private, login might fail.
Loading ESM3-open model...


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

Model loaded on device: cpu
[1/503] Embedding (len=246)...
Embedding sequence of length: 246
Embedding complete for one sequence.
[2/503] Embedding (len=272)...
Embedding sequence of length: 272
Embedding complete for one sequence.
[3/503] Embedding (len=246)...
Embedding sequence of length: 246
Embedding complete for one sequence.
[4/503] Embedding (len=246)...
Embedding sequence of length: 246
Embedding complete for one sequence.
[5/503] Embedding (len=247)...
Embedding sequence of length: 247
Embedding complete for one sequence.
[6/503] Embedding (len=227)...
Embedding sequence of length: 227
Embedding complete for one sequence.
[7/503] Embedding (len=246)...
Embedding sequence of length: 246
Embedding complete for one sequence.
[8/503] Embedding (len=249)...
Embedding sequence of length: 249
Embedding complete for one sequence.
[9/503] Embedding (len=248)...
Embedding sequence of length: 248
Embedding complete for one sequence.
[10/503] Embedding (len=241)...
Embedding sequence of 

In [ ]:
#!/usr/bin/env python

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import (
    ESM3InferenceClient,
    ESMProtein,
    ESMProteinError,
    LogitsConfig,
    LogitsOutput,
)
from typing import List, Dict, Any, Union

###########################
# 1) LOGIN TO HUGGING FACE
###########################
token = os.environ.get("HF_TOKEN", None)
if token:
    login(token=token)
else:
    print("No HF_TOKEN found in environment variables. If the model is private, login might fail.")

###########################
# 2) LOAD MODEL
###########################
print("Loading ESM3-open model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model: ESM3InferenceClient = ESM3.from_pretrained("esm3-open").to(device)
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
print("Model loaded on device:", next(model.parameters()).device)

EMBEDDING_CONFIG = LogitsConfig(
    sequence=True,          # We want sequence-level outputs (residue-level embeddings)
    return_embeddings=True, # Return final-layer embeddings
    return_hidden_states=True  # Also return all hidden states if needed
)

def embed_sequence(model: ESM3InferenceClient, sequence: str) -> Union[LogitsOutput, ESMProteinError]:
    """
    Given a raw amino acid sequence, returns a LogitsOutput containing:
      - output.embeddings  => final-layer embeddings (shape: [1, seq_len, hidden_dim])
      - output.hidden_states => list of all layer outputs, if needed
    """
    print("Embedding sequence of length:", len(sequence))
    try:
        protein = ESMProtein(sequence=sequence)
        protein_tensor = model.encode(protein)
        output = model.logits(protein_tensor, EMBEDDING_CONFIG)
        print("Embedding complete for one sequence.")
        return output
    except Exception as e:
        print("Error during embedding:", e)
        return ESMProteinError(500, str(e))

def embed_sequences_to_pt(sequences: List[str], output_pt: str = "all_embeddings.pt"):
    """
    Embeds a list of protein sequences and saves all embeddings to one .pt file.
    """
    all_embeddings = []
    meta = []

    for idx, seq in enumerate(sequences, 1):
        print(f"[{idx}/{len(sequences)}] Embedding (len={len(seq)})...")
        seq_out = embed_sequence(model, seq)
        if isinstance(seq_out, ESMProteinError):
            print(f"Error embedding sequence {idx}: {seq_out}")
            continue

        embeddings = seq_out.embeddings.detach().cpu()  # shape [1, seq_len, hidden_dim]
        all_embeddings.append(embeddings)
        meta.append({
            "index": idx,
            "input_length": len(seq),
            "embedding_shape": tuple(embeddings.shape)
        })

    torch.save({"embeddings": all_embeddings, "meta": meta}, output_pt)
    print(f"\nSaved {len(all_embeddings)} sequence embeddings to {output_pt}")

if __name__ == "__main__":
    # Import your list as `ccad_list` before running this
    # For example:
    # from somewhere import ccad_list
    # Or, for demonstration:
    embed_sequences_to_pt(loaded_AT_sequences, "ATTest_AT.pt")


No HF_TOKEN found in environment variables. If the model is private, login might fail.
Loading ESM3-open model...
Model loaded on device: cpu
[1/503] Embedding (len=287)...
Embedding sequence of length: 287
Embedding complete for one sequence.
[2/503] Embedding (len=296)...
Embedding sequence of length: 296
Embedding complete for one sequence.
[3/503] Embedding (len=298)...
Embedding sequence of length: 298
Embedding complete for one sequence.
[4/503] Embedding (len=298)...
Embedding sequence of length: 298
Embedding complete for one sequence.
[5/503] Embedding (len=299)...
Embedding sequence of length: 299
Embedding complete for one sequence.
[6/503] Embedding (len=297)...
Embedding sequence of length: 297
Embedding complete for one sequence.
[7/503] Embedding (len=282)...
Embedding sequence of length: 282
Embedding complete for one sequence.
[8/503] Embedding (len=281)...
Embedding sequence of length: 281
Embedding complete for one sequence.
[9/503] Embedding (len=291)...
Embedding s